In [149]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
from tqdm import tqdm
import math
from random import sample 
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [150]:
import warnings
warnings.filterwarnings("ignore")

In [205]:
df = pd.read_csv('vkusvill_data_20_april.csv')

In [206]:
df['date'] = df.ts.apply(lambda x: x[:10])

In [207]:
gp = pd.DataFrame(df.groupby(['order_id']).agg(list).good_foreign_id).reset_index()

indexes_to_remove = gp.good_foreign_id[gp.good_foreign_id.apply(lambda x: len(x)) == 1].index.to_list()

df = df.drop(index = indexes_to_remove).reset_index(drop = True)

In [208]:
def split_train_test(group, time_threshold):
    
    train = group[group['date'] <= time_threshold]
    test = group[group['date'] > time_threshold]
    
#     # For test, masking 20% of items
#     test_masked = test.copy()
#     test_masked['masked'] = np.random.choice([True, False], size=len(test), p=[0.2, 0.8])
#     test_masked = test_masked[test_masked['masked']]
    
    return train,test

In [336]:
df.ts.max()

'2024-04-30 02:07:27.283791'

In [340]:
time_threshold = '2024-04-29'

train_data, test_data = zip(*df.groupby('order_id').apply(split_train_test, 
                                                                 time_threshold=time_threshold))

train_data = pd.concat(train_data)
test_data = pd.concat(test_data)

In [342]:
test_data.shape

(386, 4)

In [391]:
sample_data = test_data.reset_index(drop = True)

In [392]:
sample_data

,ts,order_id,good_foreign_id,date,hsf_score,bsf_score,PSF
0,2024-04-30 00:01:35.460118,300652861999,40572,2024-04-30,1.000000,0.027363,0.319154
1,2024-04-30 00:01:35.460118,300652861999,83548,2024-04-30,0.500000,0.037313,0.176119
2,2024-04-30 00:01:35.460118,300652861999,71734,2024-04-30,0.333333,0.037313,0.126119
3,2024-04-30 00:01:35.460118,300652861999,647,2024-04-30,0.250000,-0.271891,-0.115324
4,2024-04-30 00:02:10.54555,300652862000,736,2024-04-30,1.000000,0.029809,0.320867
...,...,...,...,...,...,...,...
381,2024-04-30 02:07:27.283791,300652862096,65258,2024-04-30,0.333333,-0.425141,-0.197599
382,2024-04-30 02:07:27.283791,300652862096,22658,2024-04-30,0.250000,-0.407033,-0.209923
383,2024-04-30 02:07:27.283791,300652862096,647,2024-04-30,0.200000,-0.618610,-0.373027
384,2024-04-30 02:07:27.283791,300652862096,43207,2024-04-30,0.166667,-0.389548,-0.222684


"We introduce the personal scoring function, the neighborhood-based scoring, as well as our final scoring function, which combines them"  

History-basedscoring.𝐻𝑆𝐹(𝑥,𝑢),the history-based scoring function, accounts for the loyalty of a user to an item. In other words, it considers how frequently a user has purchased the item in the past. In addition to frequency, the recency of occurrences of an item is also important in predicting the next occurrence, as shown in [2, 4]. We define 𝐻𝑆𝐹(𝑥,𝑢) as follows:


$ HSF(x, u) = \sum_{t=0}^{|B_u| - 1} \frac{1\{x \in b_t\}}{|B_u| - t}$

where 1{𝑥 ∈b𝑡 } is equal to one if 𝑥 occurs in b𝑡 and zero otherwise. In this formulation, more recent occurrences of an item in the user history are given more weight.

In [393]:
def history_based_scoring_function(x, u, df):
    '''
    x - item_id
    u - order_id
    
    '''
    user_data = df.loc[df['order_id'] == u]
    user_data = user_data.sort_values('ts')

    score = 0
    position = 1
    for index, row in user_data.iterrows():
        if row['good_foreign_id'] == x:
            score += 1 / position
        position += 1

    return score

In [394]:
def calculate_hsf_score(row, df):
    
    x = row['good_foreign_id']
    u = row['order_id']
    score = history_based_scoring_function(x, u, df)
    
    return score

In [395]:
sample_data.columns

Index(['ts', 'order_id', 'good_foreign_id', 'date', 'hsf_score', 'bsf_score',
       'PSF'],
      dtype='object')

In [396]:
# def mapping(col):
#     return dict(zip(col.unique(), range(col.nunique())))

In [397]:
# good_foreign_id_to_item_id = mapping(sample_data["good_foreign_id"])
# sample_data["item_id"] = sample_data["good_foreign_id"].map(good_foreign_id_to_item_id)
# sample_data["basket_id"] = sample_data["order_id"].map(mapping(sample_data["order_id"]))

In [398]:
%time
sample_data['hsf_score'] = sample_data.apply(lambda row: calculate_hsf_score(row, sample_data), axis=1)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 21.9 µs


In [400]:
sample_data

,ts,order_id,good_foreign_id,date,hsf_score,bsf_score,PSF
0,2024-04-30 00:01:35.460118,300652861999,40572,2024-04-30,1.000000,0.027363,0.319154
1,2024-04-30 00:01:35.460118,300652861999,83548,2024-04-30,0.500000,0.037313,0.176119
2,2024-04-30 00:01:35.460118,300652861999,71734,2024-04-30,0.333333,0.037313,0.126119
3,2024-04-30 00:01:35.460118,300652861999,647,2024-04-30,0.250000,-0.271891,-0.115324
4,2024-04-30 00:02:10.54555,300652862000,736,2024-04-30,1.000000,0.029809,0.320867
...,...,...,...,...,...,...,...
381,2024-04-30 02:07:27.283791,300652862096,65258,2024-04-30,0.333333,-0.425141,-0.197599
382,2024-04-30 02:07:27.283791,300652862096,22658,2024-04-30,0.250000,-0.407033,-0.209923
383,2024-04-30 02:07:27.283791,300652862096,647,2024-04-30,0.200000,-0.618610,-0.373027
384,2024-04-30 02:07:27.283791,300652862096,43207,2024-04-30,0.166667,-0.389548,-0.222684


In [352]:
# sample_data.groupby(['order_id']).agg(list).good_foreign_id

**Basket-based scoring.** To compute the score of a candidate item 𝑥 given the items that are already in the current basket, we define the basket-based scoring function 𝐵𝑆𝐹(𝑥,𝑢,b𝑐). It scores an item 𝑥 based on its past co-occurrences with the items in the user’s past baskets. It takes into account three factors while scoring an item: (i) recent baskets are more informative than older baskets, (ii) recent items in the current basket are more important than older items, and (iii) the distance between the candidate item 𝑥 and the current item 𝑥𝑖 in the past baskets is important. Formally:

$ BSF(x, u, b_c) = \sum_{i=0}^{|b_c| - 1} \frac{1}{|b_c| - i} \cdot  \sum_{t=0}^{|B_u| - 1} \frac{1\{x_i \in b_t\}}{|B_u| - t} \cdot \frac{1\{x_i \in b_t\}}{|I(x, b_t) - I(x_i, b_t)|} $


In [407]:
def basket_based_scoring(candidate_item, user_baskets, all_baskets):
    
    score = 0
    
    for basket in all_baskets:
        basket_index = all_baskets.index(basket)
        if candidate_item in basket:
            factor1 = 1 / (len(all_baskets) - basket_index)
            if basket_index == len(user_baskets):
                factor2 = 0 
            else:
                factor2 = len([item for item in basket if item in user_baskets]) / (len(user_baskets) - basket_index)
            item_position = basket.index(candidate_item)
            factor3 = sum([1 / abs(item_position - basket.index(item)) for item in basket if item != candidate_item])
            score += factor1 * factor2 * factor3
    
    return score


# def BSF(x, u, b_c):
#     score = 0
#     for i in range(len(b_c)):
#         score_bc = 0
#         for t in range(len(u)):
#             count_x_in_bt = 0
#             for item in b_c[i]:
#                 if item == x:
#                     count_x_in_bt += 1
#             if count_x_in_bt > 0 and len(u) > t and len(b_c) > i and len(u) - t != 0 and len(b_c) - i != 0 and abs(len(u) - i) - len(u) + t + 1 != 0:
#                 score_bc += (1 / (len(b_c) - i)) * (1 / (len(u) - t)) * (1 / (abs(len(u) - i) - len(u) + t + 1))
#         score += score_bc
    
#     return score

In [402]:
grouped = sample_data.groupby('order_id')['good_foreign_id'].apply(list).reset_index(name='basket_items')
grouped.head()

,order_id,basket_items
0,300652861999,"[40572, 83548, 71734, 647]"
1,300652862000,"[736, 54391, 63579, 19434]"
2,300652862001,"[73010, 647, 609, 26131, 75437, 23160, 23995, ..."
3,300652862002,"[185, 647, 22658, 65890, 731, 20038, 53340]"
4,300652862004,"[51439, 24936, 82117, 647, 34217]"


In [408]:
sample_data['bsf_score'] = sample_data.apply(lambda row: basket_based_scoring(row['good_foreign_id'], 
                                                           grouped[grouped['order_id'] == row['order_id']]['basket_items'].iloc[0], 
                                                           grouped['basket_items'].tolist()), 
                                           axis=1)

In [409]:
sample_data

,ts,order_id,good_foreign_id,date,hsf_score,bsf_score,PSF
0,2024-04-30 00:01:35.460118,300652861999,40572,2024-04-30,1.000000,0.027363,0.319154
1,2024-04-30 00:01:35.460118,300652861999,83548,2024-04-30,0.500000,0.037313,0.176119
2,2024-04-30 00:01:35.460118,300652861999,71734,2024-04-30,0.333333,0.037313,0.126119
3,2024-04-30 00:01:35.460118,300652861999,647,2024-04-30,0.250000,-0.271891,-0.115324
4,2024-04-30 00:02:10.54555,300652862000,736,2024-04-30,1.000000,0.029809,0.320867
...,...,...,...,...,...,...,...
381,2024-04-30 02:07:27.283791,300652862096,65258,2024-04-30,0.333333,-0.425141,-0.197599
382,2024-04-30 02:07:27.283791,300652862096,22658,2024-04-30,0.250000,-0.407033,-0.209923
383,2024-04-30 02:07:27.283791,300652862096,647,2024-04-30,0.200000,-0.618610,-0.373027
384,2024-04-30 02:07:27.283791,300652862096,43207,2024-04-30,0.166667,-0.389548,-0.222684


Recent studies have shown the importance of users’ personal his- tory in grocery shopping [2, 7, 17]. However, they consider the next basket recommendation task as their main goal, where there is no information about a current basket b𝑢𝑛+1. For simplicity, we use the notation b𝑐 as the current basket that we aim to recommend items for as an alias for b𝑢𝑛+1. We propose the following personal scoring function𝑃𝑆𝐹(𝑥,𝑢,b𝑐)tocomputethescoreofacandidateitem𝑥 for the current incomplete basket b𝑐 of user 𝑢:


𝑃𝑆𝐹 (𝑥,𝑢, bc) = 𝛼 · 𝐻𝑆𝐹 (𝑥,𝑢) + (1 − 𝛼) · 𝐵𝑆𝐹 (𝑥,𝑢, bc),

In [410]:
def personal_scoring_function(hsf_score, bsf_score, alpha=0.3):
    
    #параметр можно с-оптимизировать потом
    combined_score = alpha * hsf_score + (1 - alpha) * bsf_score
    
    return combined_score

In [411]:
sample_data['PSF'] = sample_data.apply(lambda row: personal_scoring_function(row['hsf_score'],
                                                                   row['bsf_score']), axis=1)

In [412]:
sample_data.head(3)

,ts,order_id,good_foreign_id,date,hsf_score,bsf_score,PSF
0,2024-04-30 00:01:35.460118,300652861999,40572,2024-04-30,1.000000,0.027363,0.319154
1,2024-04-30 00:01:35.460118,300652861999,83548,2024-04-30,0.500000,0.037313,0.176119
2,2024-04-30 00:01:35.460118,300652861999,71734,2024-04-30,0.333333,0.037313,0.126119


In [414]:
sample_data

,ts,order_id,good_foreign_id,date,hsf_score,bsf_score,PSF
0,2024-04-30 00:01:35.460118,300652861999,40572,2024-04-30,1.000000,0.027363,0.319154
1,2024-04-30 00:01:35.460118,300652861999,83548,2024-04-30,0.500000,0.037313,0.176119
2,2024-04-30 00:01:35.460118,300652861999,71734,2024-04-30,0.333333,0.037313,0.126119
3,2024-04-30 00:01:35.460118,300652861999,647,2024-04-30,0.250000,-0.271891,-0.115324
4,2024-04-30 00:02:10.54555,300652862000,736,2024-04-30,1.000000,0.029809,0.320867
...,...,...,...,...,...,...,...
381,2024-04-30 02:07:27.283791,300652862096,65258,2024-04-30,0.333333,-0.425141,-0.197599
382,2024-04-30 02:07:27.283791,300652862096,22658,2024-04-30,0.250000,-0.407033,-0.209923
383,2024-04-30 02:07:27.283791,300652862096,647,2024-04-30,0.200000,-0.618610,-0.373027
384,2024-04-30 02:07:27.283791,300652862096,43207,2024-04-30,0.166667,-0.389548,-0.222684


## Neighbor-based scoring function


In addition to the personal scoring function, our model has a collaborative component that assigns scores to candidate items according to the neighboring users of the target user $ u $. Specifically, the neighbor-based scoring function $ NSF(x, u, b_c) $ is defined as follows:

$ NSF(x, u, b_c) = \frac{1}{|N_u|} \sum_{v \in N_u} \text{sim}(u, v) \text{PSF}(x, v, b_c) $

where $ N_u $ is the set of neighbors of $ u $ of size $ k $ and $ \text{sim}(u, v) $ is the similarity between users $ u $ and $ v $."

This paragraph describes how the neighbor-based scoring function is calculated in the model, taking into account the similarity between the target user $ u $ and their neighboring users.

$ \text{sim}(u, v) = \frac{\sum_{x \in I} \text{HSF}(x, u) \cdot \text{HSF}(x, v)}{\sqrt{\sum_{x \in I} \text{HSF}(x, u)^2} \cdot \sqrt{\sum_{x \in I} \text{HSF}(x, v)^2}} $

This formula calculates the cosine similarity between the user vectors $ u $ and $ v $, where $ I $ represents the item space, and $ \text{HSF}(x, u) $ denotes the history-based score of item $ x $ for user $ u $.

In [415]:
import numpy as np

def basket_similarity(basket1, basket2, dataset):
    u_scores = dataset[dataset['order_id'] == basket1]['hsf_score'].to_numpy()
    v_scores = dataset[dataset['order_id'] == basket2]['hsf_score'].to_numpy()

    max_length = max(len(u_scores), len(v_scores))
    
    #иначе проблема размерностей
    u_scores = np.pad(u_scores, (0, max_length - len(u_scores)), 'constant')
    v_scores = np.pad(v_scores, (0, max_length - len(v_scores)), 'constant')

    if len(u_scores) == 0 or len(v_scores) == 0:
        return 0  

    numerator = np.dot(u_scores, v_scores)
    denominator = np.sqrt(np.dot(u_scores, u_scores)) * np.sqrt(np.dot(v_scores, v_scores))

    return numerator / denominator if denominator != 0 else 0

In [416]:
basket_similarity(300652862001, 300652862029, sample_data)

0.6800456114499984

In [417]:
def neighbor_based_scoring_function(dataset, target_basket, k):

    similar_baskets = dataset[dataset['order_id'] != target_basket] 
    similar_baskets['similarity'] = similar_baskets['order_id'].apply(lambda x: 
                                                                       basket_similarity(target_basket, x, 
                                                                                         dataset))
    similar_baskets = similar_baskets.sort_values(by='similarity', ascending=False).head(k)


    if len(similar_baskets) > 0:
        neighbor_score = similar_baskets['similarity'].mean() * similar_baskets['PSF'].mean()
    else:
        neighbor_score = 0  

    return neighbor_score

In [418]:
k = 3

In [419]:
sample_data.head()

,ts,order_id,good_foreign_id,date,hsf_score,bsf_score,PSF
0,2024-04-30 00:01:35.460118,300652861999,40572,2024-04-30,1.000000,0.027363,0.319154
1,2024-04-30 00:01:35.460118,300652861999,83548,2024-04-30,0.500000,0.037313,0.176119
2,2024-04-30 00:01:35.460118,300652861999,71734,2024-04-30,0.333333,0.037313,0.126119
3,2024-04-30 00:01:35.460118,300652861999,647,2024-04-30,0.250000,-0.271891,-0.115324
4,2024-04-30 00:02:10.54555,300652862000,736,2024-04-30,1.000000,0.029809,0.320867


In [420]:
neighbor_based_scoring_function(sample_data, 300652861999, 3)

0.16778032681268126

**Final Scoring Function:**

The final score of a candidate item $ x $ for an incomplete basket $ b_c $ belonging to user $ u $ is calculated as:

$
\text{score}(x, u, b_c) = \beta \cdot \text{PSF}(x, u, b_c) + (1 - \beta) \cdot \text{NSF}(x, u, b_c)
$

where $ \beta $ is a hyper-parameter balancing the contributions of the personal history component and the collaborative component.


In [386]:
def final_scoring(df, psf_col, nsf_col, bets= 0.9):
    score = beta * (df.psf_col.value)

In [425]:
# sample_data.loc[0].PSF * 0.9 + (1-0.9)*neighbor_based_scoring_function(sample_data, sample_data.loc[0].order_id, 3)

In [433]:
def final_score(row, beta, neighbours_k, data):
    psf = row['PSF']
    nsf = neighbor_based_scoring_function(data, row['order_id'], neighbours_k)
    return beta * psf + (1 - beta) * nsf

In [434]:
beta = 0.9
neighbours_k = 3

In [435]:
sample_data['final_score'] = sample_data.apply(lambda row: final_score(row, beta = beta, neighbours_k = neighbours_k, data = sample_data), axis=1)


In [438]:
sample_data.groupby(['order_id']).agg(list).head(10)

,ts,good_foreign_id,date,hsf_score,bsf_score,PSF,final_score
order_id,,,,,,,
300652861999,"[2024-04-30 00:01:35.460118, 2024-04-30 00:01:...","[40572, 83548, 71734, 647]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-30]","[1.0, 0.5, 0.3333333333333333, 0.25]","[0.027363184079601987, 0.03731343283582089, 0....","[0.31915422885572137, 0.1761194029850746, 0.12...","[0.3040168386514174, 0.17528549536783528, 0.13..."
300652862000,"[2024-04-30 00:02:10.54555, 2024-04-30 00:02:1...","[736, 54391, 63579, 19434]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-30]","[1.0, 0.5, 0.3333333333333333, 0.25]","[0.029809460320823954, 0.0505050505050505, 0.0...","[0.32086662222457674, 0.18535353535353533, 0.1...","[0.3055009129044254, 0.18353913472048808, 0.13..."
300652862001,"[2024-04-30 00:02:30.62505, 2024-04-30 00:02:3...","[73010, 647, 609, 26131, 75437, 23160, 23995, ...","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-3...","[1.0, 0.047619047619047616, 0.0454545454545454...","[0.0678939297940227, -0.3692451357245388, 0.09...","[0.3475257508558159, -0.24418588072146286, 0.0...","[0.3361606692259114, -0.19637979919363946, 0.0..."
300652862002,"[2024-04-30 00:03:32.118354, 2024-04-30 00:03:...","[185, 647, 22658, 65890, 731, 20038, 53340]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-3...","[1.0, 0.5, 0.3333333333333333, 0.25, 0.2, 0.16...","[0.10478805536935296, -0.2575248599178589, -0....","[0.37335163875854704, -0.030267401942501215, 0...","[0.3466024822825833, -0.016654654348360214, 0...."
300652862004,"[2024-04-30 00:04:21.93792, 2024-04-30 00:04:2...","[51439, 24936, 82117, 647, 34217]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-3...","[1.0, 0.5, 0.3333333333333333, 0.25, 0.2]","[0.16534391534391532, 0.22486772486772486, 0.2...","[0.4157407407407407, 0.30740740740740735, 0.26...","[0.38542602401129944, 0.2879260240112994, 0.25..."
300652862005,"[2024-04-30 00:04:36.47182, 2024-04-30 00:04:3...","[74663, 647, 17037, 74472]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-30]","[1.0, 0.5, 0.3333333333333333, 0.25]","[-0.11827956989247311, -0.41338129967515924, -...","[0.21720430107526884, -0.13936690977261143, -0...","[0.21379412999993477, -0.10711995976315747, 0...."
300652862007,"[2024-04-30 00:07:57.307987, 2024-04-30 00:07:...","[39256, 89901, 22985, 647, 43202, 37206, 83613]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-3...","[1.0, 0.5, 0.3333333333333333, 0.25, 0.2, 0.16...","[0.28114754098360656, 0.37677595628415306, 0.4...","[0.4968032786885246, 0.4137431693989071, 0.387...","[0.45433033137383233, 0.3795762330131766, 0.35..."
300652862008,"[2024-04-30 00:08:38.396509, 2024-04-30 00:08:...","[51272, 66783, 54702, 18825, 47917, 647]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-3...","[1.0, 0.5, 0.3333333333333333, 0.25, 0.2, 0.16...","[-0.22833333333333333, -0.30833333333333335, -...","[0.14016666666666666, -0.06583333333333333, -0...","[0.14209933916570455, -0.04330066083429546, -0..."
300652862009,"[2024-04-30 00:10:39.924311, 2024-04-30 00:10:...","[33154, 647, 64583, 39248]","[2024-04-30, 2024-04-30, 2024-04-30, 2024-04-30]","[1.0, 0.5, 0.3333333333333333, 0.25]","[-0.031073446327683614, -0.32419321875662455, ...","[0.2782485875706215, -0.07693525312963717, 0.0...","[0.2687339878457522, -0.05093146878448062, 0.0..."


In [443]:
# sample_data.groupby(['good_foreign_id']).final_score.value_counts()